In [1]:
""" Create global HWSD2 geotif files """
import rasterio as rio
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
path_hwsd = os.path.join(os.environ['PROJDIR'], 'Soil_Properties', 'data', 'HWSD2')
src  = rio.open(os.path.join(path_hwsd, 'HWSD2.bil'))

# (1) Read map file
band1 = src.read(1)

# (2) Get lat & lon for the rectangular grid
width, height = src.width, src.height

lon = np.full(width, np.nan)
lat = np.full(height, np.nan)

for row in range(height):
    x, y = src.xy(row, 0)
    lat[row] = y

for col in range(width):
    x, y = src.xy(0, col)
    lon[col] = x

# (3) Obtain the map unit keys, excluding no data
mu_list = list(np.unique(band1.reshape(-1)))
mu_list.remove(65535)

# (4) Obtain soil property values from the MS Access Database
soildata = pd.read_csv(os.path.join(path_hwsd, 'HWSD2_LAYERS.csv'))

# List the soil layer depth
# --- only the 0-20cm and 20-40cm are relevant
layer_list = []
for layer in [f'D{i}' for i in range(1, 8)]:
    filt = soildata['LAYER'] == layer
    topdepth = soildata.loc[filt, 'TOPDEP'].iloc[0]
    botdepth = soildata.loc[filt, 'BOTDEP'].iloc[0]
    print(layer, topdepth, botdepth)
    layer_list.append(f'{topdepth}-{botdepth}cm')
print(layer_list)

# for each variable and layer, save to file
#
# Relevant variables are
# 
# COARSE    % volume    coarse fragments
# SAND      % weight    sand
# SILT      % weight    silt
# CLAY      % weight    clay
# BULK      g/cm3       bulk density
# ORG_CARBON    % weight    organic carbon content

path_out = os.path.join(os.environ['PROJDIR'], 'Soil_Moisture_v2', 'intermediate',
                        'map_predictors', 'hwsd2')

for var in ['COARSE', 'SAND', 'SILT', 'CLAY', 'BULK', 'ORG_CARBON']:
    for layer in [f'D{i}' for i in range(1, 8)]:
        print(var, layer)

        band_temp = np.full(band1.shape, np.nan)
        for mu in mu_list:
            value = soildata.query('LAYER == @layer & HWSD2_SMU_ID == @mu')[var]
            share = soildata.query('LAYER == @layer & HWSD2_SMU_ID == @mu')['SHARE']
            value = np.sum(value.values * share.values) / 100.
            band_temp = np.where(band1 == mu, value, band_temp)
        band_temp = np.where(band1 == int(src.nodata), 1e20, band_temp)

        profile = dict(src.profile)
        profile['height'] = band_temp.shape[0]
        profile['dtype'] = np.float32
        profile['nodata'] = 1e20
        file_out = os.path.join(path_out, f'{var}_{layer}.shp')
        dst = rio.open(file_out, 'w', **profile)
        dst.write(band_temp, 1)
        dst.close()

# (5) Calculate the porosity
BD_om = 1.3
BD_minerals = 2.71
BD_gravels = 2.80
vf_pores_gravels = 0.24


path_out = os.path.join(os.environ['PROJDIR'], 'Soil_Moisture_v2', 'intermediate',
                        'map_predictors', 'hwsd2')
for layer in [f'D{i}' for i in range(1, 8)]:
    # volumetric fraction
    vf_gravels_s = data_list[(layer,'COARSE')] * (1 - vf_pores_gravels) / 100
    # weight fraction of SOM within fine earth
    wf_om_fine_earth = data_list[(layer,'ORG_CARBON')] * 1.724 / 100
    # volume fraction of SOM within fine earth
    vf_om_fine_earth = np.ma.minimum(wf_om_fine_earth * data_list[(layer,'BULK')] / BD_om, 1)

    # Bulk density of soil (GRAVELS + MINERALS + ORGANIC MATTER)
    # BD is the BULK DENSITY OF FINE EARTH (MINERALS + ORGANIC MATTER)
    BD_avg = (1 - vf_gravels_s/(1 - vf_pores_gravels)) * data_list[(layer,'BULK')] + vf_gravels_s * BD_gravels

    # Mass fraction of gravels
    wf_gravels_s = vf_gravels_s * BD_gravels / BD_avg
    wf_sand_s = data_list[(layer,'SAND')] / 100 * (1 - wf_om_fine_earth) * (1 - wf_gravels_s)
    wf_silt_s = data_list[(layer,'SILT')] / 100 * (1 - wf_om_fine_earth) * (1 - wf_gravels_s)
    wf_clay_s = data_list[(layer,'CLAY')] / 100 * (1 - wf_om_fine_earth) * (1 - wf_gravels_s)
    wf_om_s = wf_om_fine_earth * (1 - wf_gravels_s)

    # Volumetric fraction of soil constituents
    vf_sand_s = wf_sand_s * BD_avg / BD_minerals
    vf_silt_s = wf_silt_s * BD_avg / BD_minerals
    vf_clay_s = wf_clay_s * BD_avg / BD_minerals
    vf_om_s = wf_om_s * BD_avg / BD_om

    # Particle density of soil (minerals + organic matter + gravels)
    BD_particle_inverse = wf_gravels_s / BD_gravels + \
        (1 - wf_gravels_s) * ((1 - wf_om_fine_earth)/BD_minerals + wf_om_fine_earth/BD_om)
    BD_particle = 1 / BD_particle_inverse

    # Porosity of soil
    vf_pores_s = 1 - BD_avg * BD_particle_inverse

    # Save to file
    profile = dict(src.profile)
    profile['height'] = band_temp.shape[0]
    profile['dtype'] = np.float32
    profile['nodata'] = 1e20
    profile['driver'] = 'GTiff'
    profile['height'] = vf_pores_s.shape[0]
    profile['dtype'] = np.float32
    profile['nodata'] = 1e20
    file_out = os.path.join(path_out, f'hwsd2_porosity_{layer}.tif')
    dst = rio.open(file_out, 'w', **profile)
    dst.write(vf_pores_s, 1)
    dst.close()

src.close()